In [ ]:
import pandas as pd
import yaml
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

from IPython.display import Markdown, display
import visualizer_helpers as vh

In [ ]:
# NOTE: SPECIALLY TAGGED PARAMETERS CELL
# Assignments in this cell may be overwritten at compile-time
# Otherwise, the below defaults to not filtering any zones
zone_set: str = 'all'
affected_tazs: list = []
affected_mazs: list = []

In [ ]:
# get corresponding filter from zone_set parameter
filter_tazs, filter_mazs = vh.get_filters(zone_set,
                                          affected_tazs=affected_tazs,
                                          affected_mazs=affected_mazs
                                        )

In [ ]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']

In [ ]:
base_trip = pd.read_csv(f"{base_dir}/final_trips.csv", 
                        index_col='trip_id', 
                        usecols=['trip_id','trip_count','trip_num','origin','destination'])

build_trip = pd.read_csv(f"{build_dir}/final_trips.csv", 
                         index_col='trip_id', 
                         usecols=['trip_id','trip_count','trip_num','origin','destination'])


In [ ]:
print(f"Filtering trips by origin and destination TAZ. Original Trips: {len(base_trip)}")
base_trip = base_trip[filter_tazs(base_trip.origin) & filter_tazs(base_trip.destination)]
build_trip = build_trip[filter_tazs(build_trip.origin) & filter_tazs(base_trip.destination)]
print(f"Trips after filtering: {len(base_trip)}")

## Trip Purpose

In [ ]:
print("Lorem ipsum") # Placeholder for CS

## Trip Destination

In [ ]:
df = base_trip[base_trip.trip_count != base_trip.trip_num].merge(build_trip[build_trip.trip_count != build_trip.trip_num],
                                                                 how='outer',
                                                                 left_index=True,
                                                                 right_index=True,
                                                                 suffixes=('_base','_build'))


In [ ]:
df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
df.loc[df.destination_build.isna(),"Case"] = "Removed"


In [ ]:
df = df.Case.value_counts()

df

In [ ]:
fig = vh.create_pie_chart(df.to_frame(),["count"])
fig.show()

## Trip Scheduling

In [ ]:
print("Lorem ipsum") # Placeholder for WSP

## Trip Mode Choice

In [ ]:
print("Lorem ipsum") # Placeholder for CS